# **Political Voting Survey Dashboard 🌏**

**Introduction**

This project aims to showcase my skills in Data Visualizations and Dashboard Development with Tableau using generated dummy data with an example case in political surveys. Also, while the Visualization using Tableau are using CSV Files, due to Tableau Public Limitations, I am also going to develop an ingestion process from Generated Data to PostgreSQL for further real-time (or updated) visualizations.

Due to current limitations in access data, I will generate dummy data using APIs from multiple random generators with a location in Indonesia. This project does not at all represent any political fact about any region and are artificial created for learning and skills showcasing purposes.

**Tables:**

- Votes (6000 Unique Values)
    - voter_name
    - voter_id
    - candidate_id
    - region_id
- Candidate (4 Unique Values)
    - Candidate_id
    - Nama (Buat 4)
    - Party_id (Buat 4)
- region (33 Unique Values)
    - region_id
    - nama_wilayah
- Electoral Vote ← Voters (dibuat dari yang menang masing-masing)
    - Wilayah (Ada 33)
    - Vote Count
    - Candidate_name
    - Party
- Party
    - party_id
    - party_name
    - member_count

In [4]:
import pandas as pd
import random
from helper.data_generate import generate_person

**Region** 

In [5]:
region_df = pd.read_csv('sources/regions_id.csv')
region_df = region_df.reset_index().rename(columns={"index": "id"})

region_df.rename(columns={"Provinsi di Indonesia":"province",
                          "Jumlah Penduduk Menurut Provinsi di Indonesia (Ribu Jiwa)":"population_count_in_thousands"},
                 inplace=True)

region_df = region_df[~region_df["province"].isin(["Indonesia"])]
region_df['id'] = region_df['id'] + 1

In [6]:
region_df

,id,province,population_count_in_thousands
0,1,Aceh,5554.8
1,2,Sumatera Utara,15588.5
2,3,Sumatera Barat,5836.2
3,4,Riau,6728.1
4,5,Kep. Riau,2183.3
5,6,Jambi,3724.3
6,7,Sumatera Selatan,8837.3
7,8,Kep. Bangka Belitung,1531.5
8,9,Bengkulu,2112.2
9,10,Lampung,9419.6


**Party** 

In [7]:
party_names = ["Partai Sosial Demokrasi Indonesia", "Partai Republik Nasionalis Nusantara", "Partai Amanat Indonesia", "Partai Kerjasama dan Utusan Rakyat"]
party_list = []
id = 1

for i in party_names:
    party = {}
    party["id"] = id
    id += 1
    party["party_name"] = i
    party["total_members"] = random.randint(1500, 5000)

    party_list.append(party)

In [8]:
party_df = pd.DataFrame(party_list)

In [9]:
party_df

,id,party_name,total_members
0,1,Partai Sosial Demokrasi Indonesia,4005
1,2,Partai Republik Nasionalis Nusantara,3183
2,3,Partai Amanat Indonesia,3216
3,4,Partai Kerjasama dan Utusan Rakyat,2247


**Candidate** 

In [10]:
generate_person()

{'gender': 'male',
 'name': {'title': 'Monsieur', 'first': 'Armin', 'last': 'Denis'},
 'location': {'street': {'number': 137, 'name': 'Avenue Debrousse'},
  'city': 'Oekingen',
  'state': 'Schwyz',
  'country': 'Switzerland',
  'postcode': 5128,
  'coordinates': {'latitude': '-27.2569', 'longitude': '-47.8051'},
  'timezone': {'offset': '+5:45', 'description': 'Kathmandu'}},
 'email': 'armin.denis@example.com',
 'login': {'uuid': '13b00a9c-2f2d-4ba8-a389-062e7d48b3f4',
  'username': 'lazymeercat129',
  'password': 'calgary',
  'salt': 'j0i2SNGy',
  'md5': 'd198256ccefc6782a5500a69f88abc3b',
  'sha1': '6f7bbb68cc1108bfc1514f6ee347992fcbf5523e',
  'sha256': 'df229b93f5b85211123300dfab5838936ea1124b127fe7f09002447b4a133547'},
 'dob': {'date': '1967-03-21T11:04:06.475Z', 'age': 58},
 'registered': {'date': '2005-06-09T16:27:52.471Z', 'age': 19},
 'phone': '079 257 19 26',
 'cell': '077 769 11 83',
 'id': {'name': 'AVS', 'value': '756.2871.3837.54'},
 'picture': {'large': 'https://randomuse

In [11]:
candidates = []

for i in party_df["id"]:
    candidate = {}
    person = generate_person()

    candidate['id'] = i
    candidate['name'] = person['name']['first'] + ' ' + person['name']['last']
    candidate['gender'] = person['gender']
    candidate['party_id'] = i

    candidates.append(candidate)

candidates_df = pd.DataFrame(candidates) 

In [12]:
candidates_df

,id,name,gender,party_id
0,1,Jasmina Lemoine,female,1
1,2,Dylan Dufour,male,2
2,3,Nando Gauthier,male,3
3,4,Pirmin Petit,male,4


**Voters** 

In [13]:
voters_csv_df = pd.read_csv("csv/voters.csv")
voters_csv_df.drop(columns=['Unnamed: 0'], inplace=True)

In [14]:
voters_csv_df

,name,gender,candidate_id,region_id
0,Mohamad Olivier,male,2,22
1,Ulrich Dumont,male,3,34
2,Catherine Petit,female,1,3
3,Lars Bourgeois,male,3,14
4,Kerstin Robert,female,4,15
...,...,...,...,...
11714,Maeva Leclerc,female,3,22
11715,Monique Blanc,female,4,8
11716,Eliana Robin,female,4,19
11717,Johan Durand,male,1,24


In [15]:
voters = []

def generate_voter():
    person = generate_person()
    voter = {}

    voter['name'] = person['name']['first'] + ' ' + person['name']['last']
    voter['gender'] = person['gender']
    voter['candidate_id'] = random.choice(candidates_df['id'].tolist())
    voter['region_id'] = random.choice(region_df['id'].tolist())

    print(voter)
    return voter


In [16]:
for x in range(7):
    for i in range(250):
        print(str(i) + " " + str(x))
        voter = generate_voter()
        voters.append(voter)

0 0
{'name': 'Claire Lopez', 'gender': 'female', 'candidate_id': 4, 'region_id': 19}
1 0
{'name': 'Noemi Colin', 'gender': 'female', 'candidate_id': 2, 'region_id': 31}
2 0
{'name': 'Flavio Brun', 'gender': 'male', 'candidate_id': 3, 'region_id': 2}
3 0
{'name': 'Livio Michel', 'gender': 'male', 'candidate_id': 2, 'region_id': 32}
4 0
{'name': 'Frieda Vincent', 'gender': 'female', 'candidate_id': 3, 'region_id': 6}
5 0
{'name': 'Rose-Marie Guillot', 'gender': 'female', 'candidate_id': 3, 'region_id': 25}
6 0
{'name': 'Nelli Denis', 'gender': 'female', 'candidate_id': 1, 'region_id': 24}
7 0
{'name': 'Michal Chevalier', 'gender': 'male', 'candidate_id': 3, 'region_id': 23}
8 0
{'name': 'Jorge Garcia', 'gender': 'male', 'candidate_id': 1, 'region_id': 32}
9 0
{'name': 'Emma Leroy', 'gender': 'female', 'candidate_id': 1, 'region_id': 10}
10 0
{'name': 'Jean-Paul Rousseau', 'gender': 'male', 'candidate_id': 3, 'region_id': 26}
11 0
{'name': 'Blaise Bonnet', 'gender': 'male', 'candidate_id'

In [17]:
voters_df = pd.DataFrame(voters)

In [18]:
voters_csv_df = pd.concat([voters_csv_df, voters_df], ignore_index=True)
voters_csv_df = voters_csv_df.drop_duplicates()
voters_csv_df.rename(columns={
    'vote': 'candidate_id'
}, inplace=True)

voters_csv_df.to_csv("csv/voters.csv")

In [20]:
voters_csv_df

,name,gender,candidate_id,region_id
0,Mohamad Olivier,male,2,22
1,Ulrich Dumont,male,3,34
2,Catherine Petit,female,1,3
3,Lars Bourgeois,male,3,14
4,Kerstin Robert,female,4,15
...,...,...,...,...
13464,Hansruedi Simon,male,1,2
13465,Delia Michel,female,2,19
13466,Van Roux,male,3,15
13467,Joel Lemoine,male,2,2


**Electoral** 

In [21]:
votes_with_candidates = pd.merge(voters_csv_df, candidates_df[['id', 'name', 'party_id']], left_on='candidate_id', right_on='id', how='inner')

votes_with_region = pd.merge(votes_with_candidates, region_df[['id', 'province']], left_on='region_id', right_on='id', how='inner')

votes_cleansed = pd.merge(votes_with_region, party_df[['id','party_name']], left_on='party_id', right_on='id', how='inner')

votes_cleansed.drop(columns=['id_x','id_y','id'], inplace=True)

votes_cleansed.rename(columns={'name_x': 'voter_name', 'name_y':'candidate_name', 'vote':'candidate_id'}, inplace=True)

votes_cleansed['vote_values'] = 1

votes_cleansed

,voter_name,gender,candidate_id,region_id,candidate_name,party_id,province,party_name,vote_values
0,Mohamad Olivier,male,2,22,Dylan Dufour,2,Sulawesi Utara,Partai Republik Nasionalis Nusantara,1
1,Ulrich Dumont,male,3,34,Nando Gauthier,3,Papua,Partai Amanat Indonesia,1
2,Catherine Petit,female,1,3,Jasmina Lemoine,1,Sumatera Barat,Partai Sosial Demokrasi Indonesia,1
3,Lars Bourgeois,male,3,14,Nando Gauthier,3,Jawa Tengah,Partai Amanat Indonesia,1
4,Kerstin Robert,female,4,15,Pirmin Petit,4,DI Yogyakarta,Partai Kerjasama dan Utusan Rakyat,1
...,...,...,...,...,...,...,...,...,...
13463,Hansruedi Simon,male,1,2,Jasmina Lemoine,1,Sumatera Utara,Partai Sosial Demokrasi Indonesia,1
13464,Delia Michel,female,2,19,Dylan Dufour,2,Kalimantan Selatan,Partai Republik Nasionalis Nusantara,1
13465,Van Roux,male,3,15,Nando Gauthier,3,DI Yogyakarta,Partai Amanat Indonesia,1
13466,Joel Lemoine,male,2,2,Dylan Dufour,2,Sumatera Utara,Partai Republik Nasionalis Nusantara,1


In [ ]:
votes_cleansed.groupby('province').agg({
    'voter_name':'count'
})

,voter_name
province,
Aceh,335
Bali,383
Banten,360
Bengkulu,373
DI Yogyakarta,335
DKI Jakarta,335
Gorontalo,354
Jambi,339
Jawa Barat,348


In [ ]:
votes_cleansed[votes_cleansed['province'] == 'Aceh']

,voter_name,gender,candidate_id,region_id,candidate_name,party_id,province,party_name
31,Timo Nguyen,male,1,1,Dominic Lefevre,1,Aceh,Partai Sosial Demokrasi Indonesia
32,Niels Robert,male,4,1,Emily Meyer,4,Aceh,Partai Kerjasama dan Utusan Rakyat
59,Louisa Pierre,female,3,1,Nelli Aubert,3,Aceh,Partai Amanat Indonesia
72,Thomas Lemoine,male,2,1,George Dubois,2,Aceh,Partai Republik Nasionalis Nusantara
73,Markus Gerard,male,1,1,Dominic Lefevre,1,Aceh,Partai Sosial Demokrasi Indonesia
...,...,...,...,...,...,...,...,...
11673,Beat David,male,4,1,Emily Meyer,4,Aceh,Partai Kerjasama dan Utusan Rakyat
11698,Lia Garnier,female,3,1,Nelli Aubert,3,Aceh,Partai Amanat Indonesia
11700,Ida Nguyen,female,3,1,Nelli Aubert,3,Aceh,Partai Amanat Indonesia
11707,Léon Fleury,male,2,1,George Dubois,2,Aceh,Partai Republik Nasionalis Nusantara


In [ ]:
# Result total pemenang
votes_cleansed.groupby('candidate_name').agg({
    'voter_name':'count'
})

,voter_name
candidate_name,
Dominic Lefevre,2851
Emily Meyer,2927
George Dubois,2969
Nelli Aubert,2972


In [22]:
votes_cleansed.to_csv('csv/votes_cleansed.csv')

In [ ]:
# Data detail untuk masing-masing provinsi
details_province = votes_cleansed.groupby(['province', 'candidate_name']).agg({
    'voter_name': ['count']
})

details_province.rename(columns={'voter_name': 'vote_count'}, inplace=True)
details_province.columns = details_province.columns.droplevel(1)
details_province.reset_index()

details_province

vote_count
province         candidate_name             
Aceh             Dominic Lefevre          85
                 Emily Meyer              86
                 George Dubois            89
                 Nelli Aubert             75
Bali             Dominic Lefevre          91
...                                      ...
Sumatera Selatan Nelli Aubert             88
Sumatera Utara   Dominic Lefevre          69
                 Emily Meyer              73
                 George Dubois            92
                 Nelli Aubert             85

[136 rows x 1 columns]

In [ ]:
# Data pememenang setiap provinsi
winners = details_province.groupby('province')['vote_count'].idxmax()
winning_candidates = details_province.loc[winners].reset_index()
winning_candidates.to_csv('csv/regional_results.csv')

winning_candidates

,province,candidate_name,vote_count
0,Aceh,George Dubois,89
1,Bali,Nelli Aubert,110
2,Banten,Dominic Lefevre,103
3,Bengkulu,George Dubois,101
4,DI Yogyakarta,Emily Meyer,97
5,DKI Jakarta,Emily Meyer,99
6,Gorontalo,Emily Meyer,111
7,Jambi,Dominic Lefevre,102
8,Jawa Barat,Nelli Aubert,95
9,Jawa Tengah,Nelli Aubert,97


In [ ]:
# Result total pemenang
winning_candidates.groupby('candidate_name').agg({
    'province':'count'
})

,province
candidate_name,
Dominic Lefevre,5
Emily Meyer,9
George Dubois,10
Nelli Aubert,10
